# Embeddings y procesamiento de texto para LLMs

Este notebook sigue el flujo del Capítulo 2 de 'Build a Large Language Model (From Scratch)' de Sebastian Raschka, adaptando el código y explicaciones en español y en mi propio estilo. Aquí se exploran los pasos clave para procesar texto y generar embeddings, fundamentales para modelos de lenguaje y sistemas agenticos.

## 1. Importar librerías necesarias

En este paso importamos las librerías fundamentales para el procesamiento de texto y la construcción de embeddings. Usaremos `torch` para las redes neuronales y `tiktoken` para la tokenización eficiente, además de utilidades estándar de Python.

> Importar correctamente estas librerías es esencial para aprovechar las capacidades modernas de procesamiento y modelado de lenguaje.

In [ ]:
import torch
import tiktoken
from torch.utils.data import Dataset, DataLoader
import os

print('torch:', torch.__version__)
print('tiktoken:', tiktoken.__version__)

## 2. Descargar y cargar el texto

En este paso cargamos el archivo 'the-verdict.txt', que contiene el texto base para el procesamiento. Tener el texto localmente permite reproducibilidad y control sobre los datos de entrada.

In [ ]:
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
print('Total de caracteres:', len(raw_text))
print(raw_text[:200])

## 3. Tokenización del texto

Utilizamos `tiktoken` para convertir el texto en una secuencia de tokens numéricos. Esta representación es la base para que el modelo pueda trabajar con el texto.

> La tokenización eficiente es clave para que el modelo entienda la estructura y el significado del texto.

In [ ]:
tokenizer = tiktoken.get_encoding('gpt2')
tokens = tokenizer.encode(raw_text)
print('Total de tokens:', len(tokens))
print('Primeros 20 tokens:', tokens[:20])

## 4. División del texto en fragmentos (chunks)

Dividimos la secuencia de tokens en fragmentos de longitud fija usando `max_length` y `stride`. El solapamiento (overlap) entre fragmentos permite que el modelo tenga acceso a contexto compartido entre diferentes partes del texto.

In [ ]:
def chunk_tokens(tokens, max_length=128, stride=64):
    chunks = []
    for i in range(0, len(tokens) - max_length + 1, stride):
        chunk = tokens[i:i+max_length]
        chunks.append(chunk)
    return chunks

max_length = 128
stride = 64
chunks = chunk_tokens(tokens, max_length, stride)
print(f'Fragmentos generados: {len(chunks)}')
print('Primer fragmento:', chunks[0][:20])

## 5. Construcción de embeddings

Para cada fragmento, generamos embeddings usando una capa de embedding de `torch`. Esto transforma los IDs de tokens en vectores densos que capturan relaciones semánticas.

In [ ]:
vocab_size = tokenizer.n_vocab
embedding_dim = 64
embedding_layer = torch.nn.Embedding(vocab_size, embedding_dim)

# Convertimos los fragmentos a tensores y generamos embeddings para el primero
tensor_chunk = torch.tensor(chunks[0], dtype=torch.long)
embeddings = embedding_layer(tensor_chunk)
print('Forma de los embeddings:', embeddings.shape)

## 6. ¿Por qué los embeddings codifican significado y cómo se relacionan con las redes neuronales?

Los embeddings transforman tokens en vectores densos que capturan relaciones semánticas. Esto es posible porque la red neuronal aprende a asignar vectores similares a palabras con significados relacionados durante el entrenamiento. Así, los embeddings permiten que el modelo entienda y procese el significado del texto de manera eficiente.

## 7. Experimento: Cambiar max_length y stride, analizar resultados

Vamos a modificar los valores de `max_length` y `stride` para ver cómo afecta la cantidad de fragmentos generados y el solapamiento entre ellos.

In [ ]:
# Probamos diferentes valores de max_length y stride
def experimenta_chunks(tokens, max_length, stride):
    chunks = chunk_tokens(tokens, max_length, stride)
    print(f"max_length={max_length}, stride={stride} => fragmentos: {len(chunks)}")
    return chunks

# Caso 1: poco solapamiento
chunks1 = experimenta_chunks(tokens, max_length=128, stride=128)
# Caso 2: mucho solapamiento
chunks2 = experimenta_chunks(tokens, max_length=128, stride=32)
# Caso 3: sin solapamiento
chunks3 = experimenta_chunks(tokens, max_length=128, stride=128)

print('Ejemplo de solapamiento (primeros 2 fragmentos, caso 2):')
print(chunks2[0][:20])
print(chunks2[1][:20])

### Análisis del experimento

Cuando el stride es pequeño, hay más fragmentos y mayor solapamiento, lo que permite que el modelo vea el mismo contexto en diferentes posiciones. Esto es útil para que el modelo aprenda dependencias largas y no pierda información relevante en los bordes de los fragmentos.

## 8. Importancia del preprocesamiento y tokenización

El preprocesamiento y la tokenización son pasos clave porque convierten el texto en una forma que puede ser entendida por el modelo. Una buena tokenización preserva la información y facilita el aprendizaje de patrones.

## 9. Utilidad de la superposición (overlap) en los fragmentos

El solapamiento entre fragmentos asegura que la información relevante que cruza los límites de los fragmentos no se pierda, permitiendo que el modelo tenga acceso a más contexto y mejore su comprensión global del texto.

## 10. Rol de los embeddings en sistemas agenticos y LLMs

En sistemas agenticos y LLMs, los embeddings son fundamentales porque permiten representar información compleja de manera compacta y manipulable, facilitando tareas como búsqueda semántica, razonamiento y generación de texto.